<a href="https://colab.research.google.com/github/rubymanderna/ML_ECGR5105/blob/main/Assignment_7/Assignment_7_1_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

b. Extend your CNN by adding one more additional convolution layer followed by an activation function and pooling function. You also need to adjust your fully connected layer properly with respect to intermediate feature dimensions. Train your network for 300 epochs. Report your training time, loss, and evaluation accuracy after 300 epochs.
Analyze your results in your report and compare your model size and accuracy over the baseline implementation in Problem 1.a. Do you see any over-fitting? Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the extended CNN architecture
class ExtendedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ExtendedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # Adjusted fully connected layer
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)  # Adjusted view based on the dimensions
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# Initialize the model, loss function, and optimizer
model = ExtendedCNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define early stopping parameters
early_stop_threshold = 5  # Number of epochs with no improvement after which training will be stopped
best_val_loss = float('inf')
epochs_no_improve = 0

# Training loop
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")


# # Training loop
# num_epochs = 300
# for epoch in range(num_epochs):
#     model.train()
#     total_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")


# Validation
model.eval()
val_loss = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        val_loss += criterion(outputs, labels).item()
accuracy = correct / total
print(f"Training Time: {num_epochs} epochs, Accuracy: {accuracy}")

avg_val_loss = val_loss / len(test_loader)
print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}, Validation Loss: {avg_val_loss}")

# Check for early stopping
if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    epochs_no_improve = 0
else:
    epochs_no_improve += 1

if epochs_no_improve == early_stop_threshold:
    print(f"Early stopping at epoch {epoch + 1} as validation loss did not improve for {early_stop_threshold} epochs.")
    break

# # Evaluation
# model.eval()
# correct = 0
# total = 0

# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = correct / total
# print(f"Training Time: {num_epochs} epochs, Accuracy: {accuracy}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.386071169544059
Epoch 2/300, Loss: 0.9639378306658372
Epoch 3/300, Loss: 0.7701364331080786
Epoch 4/300, Loss: 0.6420466435687316
Epoch 5/300, Loss: 0.5364259569655598
Epoch 6/300, Loss: 0.4451782701303587
Epoch 7/300, Loss: 0.3615442021652256
Epoch 8/300, Loss: 0.27695081382037123
Epoch 9/300, Loss: 0.22361331100068282
Epoch 10/300, Loss: 0.1715611645718441
Epoch 11/300, Loss: 0.14449689130220192
Epoch 12/300, Loss: 0.11093950741674247
Epoch 13/300, Loss: 0.10688659583416094
Epoch 14/300, Loss: 0.10143824026722204
Epoch 15/300, Loss: 0.07417517264857126
Epoch 16/300, Loss: 0.08871871888068268
Epoch 17/300, Loss: 0.07956959658702525
Epoch 18/300, Loss: 0.0726206811499851
Epoch 19/300, Loss: 0.06802677180406058
Epoch 20/300, Loss: 0.07616817888126726
Epoch 21/300, Loss: 0.06512553219149328
Epoch 22/300, Loss: 0.061692462627873625
Epoch 23/300, Loss: 0.0619336033512837
Epoch 24/300, Loss: 0.0